<a href="https://colab.research.google.com/github/cburchett/podcastcreator/blob/main/PodcastCreator_Dia_1_6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install gradio for the user interface
!pip -q install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00


In [2]:
# Install directly from GitHub
!pip -q install git+https://github.com/nari-labs/dia.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5

In [3]:
# Load the pre-trained Dia model
from dia.model import Dia

audio_model = Dia.from_pretrained("nari-labs/Dia-1.6B")

config.json:   0%|          | 0.00/941 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.44G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [4]:
# Get Google API key from userdata and set it as environment variable


import os
from google import genai

from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = API_KEY

# Create a client for Google Cloud AI API
client = genai.Client(api_key=API_KEY)

In [5]:
# Define YouTube URL
YOUTUBE_URL = "https://www.youtube.com/watch?v=rSCaiHFRx0k"

In [6]:
# Define prompt for content generation
PROMPT = """Analyze the attached Youtube video.

Based on the key topics, information, and events presented in the video, generate a medium length, conversational podcast script between two speakers, labeled S1 and S2.

The script should summarize or discuss the main points of the video in a natural, back-and-forth dialogue format.

**Crucially, format the output *exactly* as follows:**

*   Each line of dialogue must start with either `[S1]` or `[S2]`.
*   Follow the speaker tag with a space, then their dialogue.
*   Present the dialogue turns sequentially, mimicking a conversation.
*   Don't add any prefix or suffix to the conversation

**Use this specific structure as your template:**

```
[S1] {Dialogue for speaker 1}
[S2] {Dialogue for speaker 2}
[S1] {Dialogue for speaker 1, potentially a reaction or follow-up}
[S2] {Dialogue for speaker 2}
[S1] {Dialogue for speaker 1}
```

**Example of the desired output format:**

```
[S1] Hey Sam, How are you? Let me tell you about Dia it's an open weights text to dialogue model.
[S2] You get full control over scripts and voices.
[S1] Wow. Amazing. (laughs)
[S2] Try it now on Git hub or Hugging Face.
[S1] You bet I will!
```

**Constraints:**

*   Keep the turns relatively short and conversational.
*   Focus on the core message or interesting aspects of the video.
*   Adhere strictly to the `[S1]` / `[S2]` formatting.
*   **Incorporate non-verbal cues where natural and appropriate.** These should be enclosed in parentheses within the dialogue line (e.g., `(laughs)` or `(sighs)`). You may use cues from this list: `(laughs)`, `(clears throat)`, `(sighs)`, `(gasps)`, `(coughs)`, `(singing)`, `(sings)`, `(mumbles)`, `(beep)`, `(groans)`, `(sniffs)`, `(claps)`, `(screams)`, `(inhales)`, `(exhales)`, `(applause)`, `(burps)`, `(humming)`, `(sneezes)`, `(whistles)`.
*   Do not add any introductory text, explanations, or summaries outside of the formatted script itself.

**Now, analyze the video and generate the script.**

---
"""

In [19]:
TEST_SCRIPT = """
[S1] So I watched this NetApp video about data infrastructure for AI, and it really highlights how demanding these new workloads are.
[S2] Yeah, they mentioned AI data pipelines are putting serious pressure on storage architectures, right?
"""
'''
TEST_SCRIPT = """
[S1] So I watched this NetApp video about data infrastructure for AI, and it really highlights how demanding these new workloads are.
[S2] Yeah, they mentioned AI data pipelines are putting serious pressure on storage architectures, right? The requirements are constantly evolving.
[S1] Exactly. Especially for things like AI model training, which needs absolutely massive performance.
[S2] And Krish made a great point – it’s not just raw performance, but getting that performance over standard, familiar protocols like NFS or S3, using the Ethernet networks IT teams already have. (clears throat) No need to rip and replace everything.
[S1] That makes sense. Plus, the need for flexibility. The infrastructure has to handle these huge training jobs but also scale efficiently for inference later on.
[S2] And scale *independently*! They talked about scaling storage, compute, and networking separately to really maximise how resources are used.
[S1] That ties into what Russell was saying too, about fitting AI into the *existing* data center. (laughs) You can't just build new ones overnight.
[S2] Right! So the underlying storage needs to be super dense and energy-efficient to work within current power and space limits. It has to integrate, not be its own isolated thing.
[S1] It sounds like IT Ops needs solutions that work with their current tools and skills, rather than something completely alien. AI is becoming just another workload, albeit a demanding one.
[S2] Absolutely. And it's often hybrid – data and tools might be on-prem *and* in the cloud, maybe even multiple clouds.
[S1] So you need that seamless experience, right? Moving data easily, managing it consistently wherever it lives, breaking down those silos.
[S2] Yeah, defying "data gravity" as Krish put it. You need that consistent view and operation across your whole environment, on-prem and cloud(s).
[S1] It's really moving beyond just playing around with AI in a lab, isn't it? This is about running it in production, integrated with real business processes.
[S2] Totally. That means it needs to be reliable, manageable, automatable – all the things you expect from production infrastructure. The days of just POCs are evolving fast.
[S1] So the key takeaway seems to be needing a really flexible, high-performance, but operationally practical data infrastructure that spans wherever AI workloads run.
[S2] Definitely. It needs to be ready for today's AI demands and whatever comes next, without forcing IT teams to start from scratch.
"""
'''

'\nTEST_SCRIPT = """\n[S1] So I watched this NetApp video about data infrastructure for AI, and it really highlights how demanding these new workloads are.\n[S2] Yeah, they mentioned AI data pipelines are putting serious pressure on storage architectures, right? The requirements are constantly evolving.\n[S1] Exactly. Especially for things like AI model training, which needs absolutely massive performance.\n[S2] And Krish made a great point – it’s not just raw performance, but getting that performance over standard, familiar protocols like NFS or S3, using the Ethernet networks IT teams already have. (clears throat) No need to rip and replace everything.\n[S1] That makes sense. Plus, the need for flexibility. The infrastructure has to handle these huge training jobs but also scale efficiently for inference later on.\n[S2] And scale *independently*! They talked about scaling storage, compute, and networking separately to really maximise how resources are used.\n[S1] That ties into what 

In [8]:
# Function to create a new folder with a timestamp

import os
from datetime import datetime

def ensure_folder_exists():
    now = datetime.now()
    timestamp = now.strftime('%Y-%m-%d %H:%M:%S')
    folder_path = '/content/' + timestamp
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")
    return folder_path

In [9]:
# Function to generate podcast script using Google Cloud AI API

from google.genai import types

def generate_podcast_script(youtube_url, model, prompt):
    response = client.models.generate_content(
        model=model,
        contents=types.Content(
            parts=[
                types.Part(text=prompt),
                types.Part(
                    file_data=types.FileData(file_uri=youtube_url)
                )
            ]
        )
    )
    return response.text

In [10]:
# Function to split podcast transcript into segments

import re

def split_podcast_transcript(transcript, pairs):
    """Splits a podcast transcript into segments based on S1 and S2 pairs.

    Args:
        transcript: The podcast transcript as a string.

    Returns:
        A list of strings, where each string represents a segment of the transcript.
        Returns an empty list if the input is invalid or no valid segments are found.
    """

    segments = []
    try:
        # Split the transcript into lines
        lines = transcript.strip().split('\n')

        # Use regular expressions to find S1 and S2 pairs
        pattern = r"\[(S[12])\](.*)"
        s1_s2_pairs = []
        for line in lines:
          match = re.match(pattern, line)
          if match:
            s1_s2_pairs.append(match.groups())

        for i in range(0, len(s1_s2_pairs), pairs):
            segment = ""
            for j in range(i, min(i + pairs, len(s1_s2_pairs))):
                segment += f"[{s1_s2_pairs[j][0]}] {s1_s2_pairs[j][1]}\n"
            segments.append(segment.strip())
    except Exception as e:
        print(f"Error processing transcript: {e}")
        return []

    return segments

In [11]:
# Function to extract podcast numbering from filename
# Function to combine multiple MP3 files into one
import os
import soundfile as sf
from pydub import AudioSegment

def extract_podcast_numbering(filename):
    # Extract the numeric part at the beginning of the filename
    match = re.match(r'podcast_(\d+)\.mp3', filename)
    return int(match.group(1)) if match else float('inf')

def combine_mp3s(folder_path, output_file):
    """Combines all MP3 files in a folder into a single MP3 file.

    Args:
        folder_path: The path to the folder containing the MP3 files.
        output_file: The path to the output MP3 file.
    """
    combined = AudioSegment.empty()
    file_list = [f for f in os.listdir(folder_path) if f.endswith(".mp3")]
    file_list.sort(key=extract_podcast_numbering)
    print("Sort Order:", file_list)
    for filename in file_list:
        if filename.endswith(".mp3"):
            filepath = os.path.join(folder_path, filename)
            try:
                segment = AudioSegment.from_mp3(filepath)
                combined += segment
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    combined.export(output_file, format="mp3")
    print(f"Combined {len(file_list)} MP3 files into {output_file}")
    return output_file

In [12]:
def write_formatted_script_to_file(script_content, output_file_path):
    """
    Writes a formatted Python script to a file for clear reading.

    Args:
        script_content (str): The content of the Python script as a string.
        output_file_path (str): The path to the output file where the script will be written.
    """
    # Split the script content into lines
    lines = script_content.split('\n')

    # Format the lines for clear reading
    formatted_lines = []
    for line in lines:
        # Strip leading and trailing whitespace
        stripped_line = line.strip()
        # Add the formatted line to the list, ensuring proper indentation
        if stripped_line:
            formatted_lines.append(stripped_line)

    # Join the formatted lines back into a single string with proper line breaks
    formatted_script = '\n'.join(formatted_lines)

    # Write the formatted script to the output file
    with open(output_file_path, 'w') as file:
        file.write(formatted_script)

    print(f"Formatted script has been written to {output_file_path}")

In [13]:
# Function to set random seed for reproducibility

import random
import numpy as np
import torch

def set_seed(seed: int):
    """Sets the random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [14]:
import time
import torch

def generate_podcast_segment(
      seg: str,
      idx: int,
      max_new_tokens: int,
      cfg_scale: float,
      temperature: float,
      top_p: float,
      cfg_filter_top_k: int,
      folder_path: str
    ):

    print(f"Generating segment {idx+1}")
    print(seg)
    start_time = time.time()
    # Use torch.inference_mode() context manager for the generation call
    with torch.inference_mode():
      output = audio_model.generate(
          text=seg,
          max_tokens=max_new_tokens,
          cfg_scale=cfg_scale,
          temperature=temperature,
          top_p=top_p,
          cfg_filter_top_k=cfg_filter_top_k
        )
    sf.write(folder_path + f"/podcast_{idx+1}.mp3", output, 44100)
    end_time = time.time()
    print(f"Generation finished in {end_time - start_time:.2f} seconds.\n")

In [18]:
# Function to generate podcast audio

import soundfile as sf

def generate_podcast_segments(
      transcript: str,
      pairs: int,
      max_new_tokens: int,
      cfg_scale: float,
      temperature: float,
      top_p: float,
      cfg_filter_top_k: int,
      speed_factor: float,
      folder_path: str,
      seed: int = 0,
    ):

    # Set and Display Generation Seed
    if seed is None or seed < 0:
        seed = random.randint(0, 2 ** 32 - 1)
        print(f"\nNo seed provided, generated random seed: {seed}\n")
    else:
        print(f"\nUsing user-selected seed: {seed}\n")
    set_seed(seed)

    # Split transcript into segments
    segments = split_podcast_transcript(transcript, pairs)
    print(f"Number of segments: " + str(len(segments)))

    output_data = [] # List to store tuples of (segment text, audio path)

    # Generate audio for each segment
    for idx, seg in enumerate(segments):
      generate_podcast_segment(seg, idx,
                               max_new_tokens,
                               cfg_scale,
                               temperature,
                               top_p,
                               cfg_filter_top_k,
                               folder_path)


      audio_path = folder_path + f"/podcast_{idx+1}.mp3"

    output_data.append((seg, audio_path)) # Append segment text and audio path

    return output_data # Return the list of data and the folder path

In [16]:
def generate_final_podcast(
        FOLDER_PATH: str,
        transcript: str
        ):

        # Print out transcript
        output_file_path = FOLDER_PATH + "/final_transcript.txt"
        write_formatted_script_to_file(transcript, output_file_path)
        print(f"Transcript written to {output_file_path}")

        # Combine audio segments into final podcast
        return combine_mp3s(FOLDER_PATH, FOLDER_PATH + "/final_podcast.mp3")

In [21]:
# Create Gradio interface for podcast generation
import gradio as gr


with gr.Blocks() as podcast_script_generator:
  segments_state = gr.State([])
  gr.Markdown("## Podcast Generator")
  with gr.Tab("Script"):
    with gr.Row():
      with gr.Column():
        youtube_url = gr.Textbox(label="URL", value=YOUTUBE_URL)
        selected_model = gr.Dropdown(["gemini-2.5-pro-exp-03-25"], label="Transcript model")
        system_prompt = gr.Textbox(label="System prompt", value=PROMPT)
        generate_script_button = gr.Button("Generate script")
      with gr.Column():
        podcast_script = gr.Textbox(label="Podcast script", lines=20, value=TEST_SCRIPT)  # Remove val when move to production
  generate_script_button.click(fn=generate_podcast_script, inputs=[youtube_url, selected_model, system_prompt], outputs=[podcast_script])

  with gr.Tab("Podcast"):
     with gr.Row():
       with gr.Column():
          final_podcast_script = gr.Textbox(label="Final podcast script", lines=20, value=TEST_SCRIPT) # Remove val when move to production
          pairs = gr.Slider(
                    label="Segment pairs",
                    minimum=1,
                    maximum=10,
                    value=1,  # Default
                    step=1,
                    info="Higher values increase number of [S1] [S2] pairs will be in each batch.",
                )
          with gr.Accordion("Generation Parameters", open=False):
                max_new_tokens = gr.Slider(
                    label="Max New Tokens (Audio Length)",
                    minimum=860,
                    maximum=3072,
                    value=audio_model.config.data.audio_length,  # Use config default if available, else fallback
                    step=50,
                    info="Controls the maximum length of the generated audio (more tokens = longer audio).",
                )
                cfg_scale = gr.Slider(
                    label="CFG Scale (Guidance Strength)",
                    minimum=1.0,
                    maximum=5.0,
                    value=3.0,  # Default
                    step=0.1,
                    info="Higher values increase adherence to the text prompt.",
                )
                temperature = gr.Slider(
                    label="Temperature (Randomness)",
                    minimum=1.0,
                    maximum=1.5,
                    value=1.1,  # Default
                    step=0.05,
                    info="Lower values make the output more deterministic, higher values increase randomness.",
                )
                top_p = gr.Slider(
                    label="Top P (Nucleus Sampling)",
                    minimum=0.80,
                    maximum=1.0,
                    value=0.95,  # Default
                    step=0.01,
                    info="Filters vocabulary to the most likely tokens cumulatively reaching probability P.",
                )
                cfg_filter_top_k = gr.Slider(
                    label="CFG Filter Top K",
                    minimum=15,
                    maximum=50,
                    value=30,
                    step=1,
                    info="Top k filter for CFG guidance.",
                )
                speed_factor_slider = gr.Slider(
                    label="Speed Factor",
                    minimum=0.8,
                    maximum=1.0,
                    value=0.84,
                    step=0.02,
                    info="Adjusts the speed of the generated audio (1.0 = original speed).",
                )
                seed_input = gr.Number(
                    label="Generation Seed (Optional)",
                    value=-1,
                    precision=0,  # No decimal points
                    step=1,
                    interactive=True,
                    info="Set a generation seed for reproducible outputs. Leave empty or -1 for random seed.",
                )
          with gr.Row():
            generated_filepath = gr.Textbox(label="File Path")
            generate_path_button = gr.Button("Generate path")
          generate_podcast_segments_button = gr.Button("Generate podcast segments")
       with gr.Column():

         with gr.Accordion("Segments", open=False) as all_segments_accordion:
             segments_container = gr.Column()

         generate_final_podcast_button = gr.Button("Generate final podcast")
         final_podcast_audio = gr.Audio(label="Final podcast audio", type='filepath')

  generate_path_button.click(fn=ensure_folder_exists, outputs=[generated_filepath])

  def update_segments_in_accordion(output_data):
    # This function creates the components to display inside the accordion
    components = []
    for i, (text, audio_path) in enumerate(output_data):
        components.append(gr.Textbox(label=f"Segment {i+1} Text", value=text, interactive=False))
        components.append(gr.Audio(label=f"Segment {i+1} Audio", value=audio_path, type='filepath'))
    return components # Return a list of components

  generate_podcast_segments_button.click(
      fn=generate_podcast_segments, # Call the generation function and pass its output
      inputs=[
            final_podcast_script,
            pairs,
            max_new_tokens,
            cfg_scale,
            temperature,
            top_p,
            cfg_filter_top_k,
            speed_factor_slider,
            generated_filepath,
            seed_input,
      ],
      outputs=[segments_state]
  ).then(
        fn=update_segments_in_accordion, # Call the update function with the generation output
        inputs=[segments_state], # Get the output from the previous function call
        outputs=[segments_container] # Update the segments container
  )

  generate_final_podcast_button.click(fn=generate_final_podcast, inputs=[
     generated_filepath,
     final_podcast_script
  ], outputs=[final_podcast_audio])

  # Used to auto-update the final_podcast_script textbox with the value generated on the previous tab
  podcast_script.change(fn=lambda x: x, inputs=podcast_script, outputs=final_podcast_script)

podcast_script_generator.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1fbe4268e5a990fc11.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Folder '/content/2025-05-25 16:22:42' created.

No seed provided, generated random seed: 3390997336

Number of segments: 2
Generating segment 1
[S1]  So I watched this NetApp video about data infrastructure for AI, and it really highlights how demanding these new workloads are.
Generation finished in 53.05 seconds.

Generating segment 2
[S2]  Yeah, they mentioned AI data pipelines are putting serious pressure on storage architectures, right?
Generation finished in 25.88 seconds.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2201, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1978, in postprocess_data
    raise InvalidComponentError(
gradio.exceptions.InvalidComponentError: <class 'gradio.layouts.column.Column'> Component not a valid output component.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1fbe4268e5a990fc11.gradio.live
